### データの取り扱い
PyTorchではデータを取り扱う際に、基本的な要素が2つ存在します。  
torch.utils.data.Datasetとtorch.utils.data.DataLoaderです。

In [1]:
%matplotlib inline

In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose
import matplotlib.pyplot as plt

In [4]:
# 訓練データをdatasetsからダウンロード
training_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# テストデータをdatasetsからダウンロード
test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

In [5]:
batch_size = 64

# データローダーの作成
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print("Shape of X [N, C, H, W]: ", X.shape)
    print("Shape of y: ", y.shape, y.dtype)
    break

Shape of X [N, C, H, W]:  torch.Size([64, 1, 28, 28])
Shape of y:  torch.Size([64]) torch.int64


### モデルの構築
PyTorchでニューラルネットワークの形を定義する際には、nn.Moduleを継承します。  
__init__ 関数で、ネットワークの各レイヤーを定義し、データの順伝搬をforward 関数に定義します。  
なお処理を高速化するために、可能であればニューラルネットワークをGPU上へ移動させます。

In [6]:
# 訓練に際して、可能であればGPU（cuda）を設定します。GPUが搭載されていない場合はCPUを使用します
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Using {} device".format(device))

# modelを定義します
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU()
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
)


### モデルパラメータの最適化
ニューラルネットワークモデルを訓練するためには、  
損失関数：loss functionと、最適化手法：optimizerが必要です。

In [7]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [8]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)
        
        # 損失誤差を計算
        pred = model(X)
        loss = loss_fn(pred, y)
        
        # バックプロパゲーション
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        if batch % 100 == 0:
            loss, current = loss.item(), batch * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [9]:
def test(dataloader, model):
    size = len(dataloader.dataset)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= size
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [14]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model)
print("Done!")

Epoch 1
-------------------------------
loss: 2.030451  [    0/60000]
loss: 1.991876  [ 6400/60000]
loss: 2.151433  [12800/60000]
loss: 2.032781  [19200/60000]
loss: 2.076157  [25600/60000]
loss: 2.048995  [32000/60000]
loss: 2.074563  [38400/60000]
loss: 2.166357  [44800/60000]
loss: 2.097828  [51200/60000]
loss: 2.122299  [57600/60000]
Test Error: 
 Accuracy: 41.2%, Avg loss: 0.032575 

Epoch 2
-------------------------------
loss: 2.030451  [    0/60000]
loss: 1.991876  [ 6400/60000]
loss: 2.151433  [12800/60000]
loss: 2.032781  [19200/60000]
loss: 2.076157  [25600/60000]
loss: 2.048995  [32000/60000]
loss: 2.074563  [38400/60000]
loss: 2.166357  [44800/60000]
loss: 2.097828  [51200/60000]
loss: 2.122299  [57600/60000]
Test Error: 
 Accuracy: 41.2%, Avg loss: 0.032575 

Epoch 3
-------------------------------
loss: 2.030451  [    0/60000]
loss: 1.991876  [ 6400/60000]
loss: 2.151433  [12800/60000]
loss: 2.032781  [19200/60000]
loss: 2.076157  [25600/60000]
loss: 2.048995  [32000/600

### モデルの保存
モデルを保存する一般的な方法は、モデルの内部状態の辞書（モデルのパラメータを含む）をシリアル化する手法です。

In [15]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


### モデルの読み込み
モデルの読み込む際には、まずモデルの構造を再作成し、そのインスタンスに、保存しておいた状態辞書をロードします。

In [16]:
model = NeuralNetwork()
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [17]:
model.eval()
for idx in range(20):
    x, y = test_data[idx][0], test_data[idx][1]
    with torch.no_grad():
        pred = model(x)
        predicted, actual = pred[0].argmax(0), y
        print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "4", Actual: "7"
Predicted: "3", Actual: "2"
Predicted: "1", Actual: "1"
Predicted: "0", Actual: "0"
Predicted: "4", Actual: "4"
Predicted: "1", Actual: "1"
Predicted: "4", Actual: "4"
Predicted: "4", Actual: "9"
Predicted: "0", Actual: "5"
Predicted: "4", Actual: "9"
Predicted: "0", Actual: "0"
Predicted: "0", Actual: "6"
Predicted: "4", Actual: "9"
Predicted: "0", Actual: "0"
Predicted: "1", Actual: "1"
Predicted: "3", Actual: "5"
Predicted: "4", Actual: "9"
Predicted: "3", Actual: "7"
Predicted: "3", Actual: "3"
Predicted: "4", Actual: "4"
